# 03 — Модель и бэктест (валидация + диагностика)

Порог и полярность подбираются только на ВАЛИДАЦИИ, тест — финальная честная оценка.
Диагностика trade_count и экспозиции помогает понять, почему сделок мало.

Полярность (pred vs -pred) рассматриваем как гиперпараметр,
поэтому выбираем её на валидации. Порог также фиксируется по валидации.
Это защищает от утечки информации из теста.

In [1]:
from pathlib import Path
import json
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().resolve()
if (PROJECT_ROOT / "src").exists():
    ROOT = PROJECT_ROOT
elif (PROJECT_ROOT.parent / "src").exists():
    ROOT = PROJECT_ROOT.parent
else:
    ROOT = PROJECT_ROOT

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.features import get_feature_columns, drop_na_for_training
from src.model import TrainConfig, StandardScaler, train_mlp_model, predict, compute_regression_metrics
from src.backtest import backtest_long_short, backtest_long_short_horizon

DATA_PATH = ROOT / "data" / "eurusd_features.parquet"
ARTIFACT_DIR = ROOT / "data" / "artifacts"
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(DATA_PATH)
df = drop_na_for_training(df)
df = df.sort_values("time").reset_index(drop=True)


In [2]:
feature_cols = get_feature_columns()
X = df[feature_cols].values
y = df["target"].values

assert np.isfinite(X).all(), "NaNs or infs in features"
assert df["time"].is_monotonic_increasing, "Time must be sorted"


In [3]:
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

X_train, y_train = X[:train_end], y[:train_end]
X_val, y_val = X[train_end:val_end], y[train_end:val_end]
X_test, y_test = X[val_end:], y[val_end:]

df_val = df.iloc[train_end:val_end].reset_index(drop=True)
df_test = df.iloc[val_end:].reset_index(drop=True)


In [4]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)


In [5]:
cfg = TrainConfig(epochs=200, batch_size=1024, lr=1e-3, weight_decay=1e-4, patience=5)
model, history = train_mlp_model(X_train_s, y_train, X_val_s, y_val, cfg)

pred_val = predict(model, X_val_s)
pred_test = predict(model, X_test_s)


In [6]:
bt_pos = backtest_long_short_horizon(df_val.assign(pred=pred_val), threshold=0.0, hold_bars=3)
bt_neg = backtest_long_short_horizon(df_val.assign(pred=-pred_val), threshold=0.0, hold_bars=3)

polarity = 1 if bt_pos.metrics["sharpe"] >= bt_neg.metrics["sharpe"] else -1
print("VAL Sharpe pos:", bt_pos.metrics["sharpe"], "neg:", bt_neg.metrics["sharpe"])
print("Chosen polarity:", polarity)

pred_val_final = polarity * pred_val
pred_test_final = polarity * pred_test


VAL Sharpe pos: -4.699676110397464 neg: -11.06478784802914
Chosen polarity: 1


In [7]:
abs_pred = np.abs(pred_val_final)
quantiles = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95]
thresholds = sorted(set([float(np.quantile(abs_pred, q)) for q in quantiles]))

regimes = [None, "adx", "h1_align", "adx_and_h1"]
sizing_modes = ["discrete", "continuous"]
target_ann_vols = [None, 0.08, 0.12]

val_rows = []
for th in thresholds:
    for reg in regimes:
        for sm in sizing_modes:
            for tav in target_ann_vols:
                if sm == "discrete" and tav is not None:
                    continue
                bt = backtest_long_short_horizon(
                    df_val.assign(pred=pred_val_final),
                    threshold=th,
                    hold_bars=3,
                    cost_bps=0.5,
                    regime=reg,
                    sizing_mode=sm,
                    target_ann_vol=tav,
                )
                m = bt.metrics
                d = bt.debug
                if m["trade_count"] >= 50 and d["consistency_abs"] > 1e-3:
                    print("WARNING: consistency_abs too large", th, reg, sm, tav, d)
                    continue
                val_rows.append(
                    {
                        "threshold": th,
                        "regime": reg,
                        "sizing_mode": sm,
                        "target_ann_vol": tav,
                        "sharpe": m["sharpe"],
                        "total_return": m["total_return"],
                        "max_drawdown": m["max_drawdown"],
                        "trade_count": m["trade_count"],
                        "hit_rate": m["hit_rate"],
                        "payoff_ratio": m["payoff_ratio"],
                        "monthly_return_est": m["monthly_return_est"],
                        "consistency_abs": d["consistency_abs"],
                    }
                )

val_table = pd.DataFrame(val_rows)
val_table


,threshold,regime,sizing_mode,target_ann_vol,sharpe,total_return,max_drawdown,trade_count,hit_rate,payoff_ratio,monthly_return_est,consistency_abs
0,0.000151,None,discrete,NaN,-8.508415,-0.134731,-0.138935,1409.0,0.419446,0.968444,-0.040253,2.803313e-15
1,0.000151,None,continuous,NaN,-8.060952,-0.048868,-0.050111,1409.0,0.419446,0.969306,-0.014124,7.591150e-15
2,0.000151,None,continuous,0.08,-8.170254,-0.062654,-0.063931,1409.0,0.419446,0.943029,-0.018202,5.453971e-15
3,0.000151,None,continuous,0.12,-8.168763,-0.092531,-0.094383,1409.0,0.419446,0.943149,-0.027190,8.298917e-15
4,0.000151,adx,discrete,NaN,-5.826870,-0.071521,-0.081407,615.0,0.421138,0.918919,-0.020848,2.928213e-15
...,...,...,...,...,...,...,...,...,...,...,...,...
155,0.000796,h1_align,continuous,0.12,0.210865,0.001456,-0.008201,76.0,0.434211,1.367462,0.000413,6.665675e-16
156,0.000796,adx_and_h1,discrete,NaN,0.345922,0.000599,-0.002808,27.0,0.481481,1.198174,0.000170,1.904943e-16
157,0.000796,adx_and_h1,continuous,NaN,0.345922,0.000599,-0.002808,27.0,0.481481,1.198174,0.000170,1.904943e-16
158,0.000796,adx_and_h1,continuous,0.08,0.487303,0.001149,-0.003369,27.0,0.481481,1.253335,0.000326,1.100899e-15


In [8]:
valid = val_table[val_table["trade_count"] >= 50]
valid = valid[valid["max_drawdown"] >= -0.05]
if len(valid) == 0:
    print("WARNING: No valid configs; using first row")
    best_row = val_table.iloc[0]
else:
    valid = valid.sort_values("monthly_return_est", ascending=False)
    valid = valid.sort_values("sharpe", ascending=False)
    valid = valid.sort_values("max_drawdown", ascending=False)
    best_row = valid.iloc[0]

best_threshold = float(best_row["threshold"])
best_regime = best_row["regime"]
best_sizing_mode = best_row["sizing_mode"]
best_target_ann_vol = best_row["target_ann_vol"]
print("Selected (VAL):", best_threshold, best_regime, best_sizing_mode, best_target_ann_vol)


Selected (VAL): 0.0003133857389912009 adx_and_h1 continuous nan


In [9]:
metrics_val = compute_regression_metrics(pred_val_final, y_val)
metrics_test = compute_regression_metrics(pred_test_final, y_test)

bt_test = backtest_long_short_horizon(
    df_test.assign(pred=pred_test_final),
    threshold=best_threshold,
    hold_bars=3,
    cost_bps=0.5,
    regime=best_regime,
    sizing_mode=best_sizing_mode,
    target_ann_vol=best_target_ann_vol,
)

if bt_test.metrics["trade_count"] > 0:
    assert bt_test.debug["consistency_abs"] < 1e-5

print("VAL metrics:", metrics_val)
print("TEST metrics:", metrics_test)
print("TEST backtest metrics:", bt_test.metrics)
print("TEST debug:", {
    "equity_log": bt_test.debug["equity_log"],
    "trade_log_sum": bt_test.debug["trade_log_sum"],
    "consistency_abs": bt_test.debug["consistency_abs"],
    "nonzero_pos_bars": bt_test.debug["nonzero_pos_bars"],
})
print("Monthly return est (TEST):", bt_test.metrics["monthly_return_est"])

bt_test.trades.head(10)


AssertionError: 

In [ ]:
baseline_bt = backtest_long_short(df_test.assign(pred=0.0), threshold=0.0)
print("Baseline pred=0 metrics:", baseline_bt.metrics)


Печатаем критерии остановки и возможные причины провала.
Если trade_count низкий, вероятно порог слишком высокий или предсказания почти нулевые.

In [ ]:
criteria = {
    "sharpe_ok": bt_test.metrics["sharpe"] >= 0.30,
    "maxdd_ok": bt_test.metrics["max_drawdown"] >= -0.12,
    "trades_ok": bt_test.metrics["trade_count"] >= 50,
    "diracc_ok": metrics_test["dir_acc"] >= 0.51,
}
print("PASS/FAIL:", criteria)

if not all(criteria.values()):
    print("Likely causes:")
    if bt_test.metrics["trade_count"] < 50:
        print("1) Too few trades: threshold too high or predictions near zero.")
    if bt_test.debug.get("pred_abs_p95", 0) < best_threshold:
        print("2) Threshold above prediction scale; decrease quantiles.")
    if bt_test.debug.get("signal_counts", {}).get("long", 0) == 0 or bt_test.debug.get("signal_counts", {}).get("short", 0) == 0:
        print("3) Model predicts mostly one sign; consider polarity or regularization.")

results_payload = {
    "polarity": polarity,
    "threshold": best_threshold,
    "regime": best_regime,
    "sizing_mode": best_sizing_mode,
    "target_ann_vol": best_target_ann_vol,
    "val_metrics": metrics_val,
    "test_metrics": metrics_test,
    "test_backtest_metrics": bt_test.metrics,
}
with (ARTIFACT_DIR / "results.json").open("w", encoding="utf-8") as f:
    json.dump(results_payload, f, ensure_ascii=False, indent=2)
